## Embeddings for Weather Data

An embedding is a low-dimensional, vector representation of a (typically) high-dimensional feature which maintains the semantic meaning of the feature in a such a way that similar features are close in the embedding space.

In this notebook, we use autoencoders to create embeddings for HRRR images. We can then use the embeddings to search for "similar" weather patterns.

In [ ]:
!sudo apt-get -y --quiet install libeccodes0

In [ ]:
%pip install -q cfgrib xarray pydot

In [ ]:
import apache_beam as beam
print(beam.__version__)

### Reading HRRR data and converting to TensorFlow Records

HRRR data comes in a Grib2 files on Cloud Storage.

In [ ]:
!gsutil ls -l gs://high-resolution-rapid-refresh/hrrr.20200811/conus/hrrr.*.wrfsfcf00*

In [ ]:
FILENAME="gs://high-resolution-rapid-refresh/hrrr.20200811/conus/hrrr.t18z.wrfsfcf06.grib2"   # derecho in the Midwest
!gsutil ls -l {FILENAME}

In [ ]:
import xarray as xr
import tensorflow as tf
import tempfile
import cfgrib

with tempfile.TemporaryDirectory() as tmpdirname:
    TMPFILE="{}/read_grib".format(tmpdirname)
    tf.io.gfile.copy(FILENAME, TMPFILE, overwrite=True)
    ds = cfgrib.open_datasets(TMPFILE)
    print(ds)

We have to choose one of the following:
```
    filter_by_keys={'typeOfLevel': 'unknown'}
    filter_by_keys={'typeOfLevel': 'cloudTop'}
    filter_by_keys={'typeOfLevel': 'surface'}
    filter_by_keys={'typeOfLevel': 'heightAboveGround'}
    filter_by_keys={'typeOfLevel': 'isothermal'}
    filter_by_keys={'typeOfLevel': 'isobaricInhPa'}
    filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'}
    filter_by_keys={'typeOfLevel': 'sigmaLayer'}
    filter_by_keys={'typeOfLevel': 'meanSea'}
    filter_by_keys={'typeOfLevel': 'heightAboveGroundLayer'}
    filter_by_keys={'typeOfLevel': 'sigma'}
    filter_by_keys={'typeOfLevel': 'depthBelowLand'}
    filter_by_keys={'typeOfLevel': 'isobaricLayer'}
    filter_by_keys={'typeOfLevel': 'cloudBase'}
    filter_by_keys={'typeOfLevel': 'nominalTop'}
    filter_by_keys={'typeOfLevel': 'isothermZero'}
    filter_by_keys={'typeOfLevel': 'adiabaticCondensation'}
```

In [ ]:
import xarray as xr
import tensorflow as tf
import tempfile
import cfgrib
import numpy as np

refc = 0
with tempfile.TemporaryDirectory() as tmpdirname:
    TMPFILE="{}/read_grib".format(tmpdirname)
    tf.io.gfile.copy(FILENAME, TMPFILE, overwrite=True)
    #ds = xr.open_dataset(TMPFILE, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'surface', 'stepType': 'instant'}})
    #ds.data_vars['prate'].plot()  # crain, prate
    ds = xr.open_dataset(TMPFILE, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'unknown', 'stepType': 'instant'}})
    #ds = xr.open_dataset(TMPFILE, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'atmosphere', 'stepType': 'instant'}})
    refc = ds.data_vars['refc']
    refc.plot()
    print(np.array([refc.sizes['y'], refc.sizes['x']]))
    print(refc.time.data)
    print(refc.valid_time.data)

In [ ]:
print(str(refc.time.data)[:19])

In [ ]:
import numpy as np

def _array_feature(value, min_value, max_value):
    """Wrapper for inserting ndarray float features into Example proto."""
    value = np.nan_to_num(value.flatten()) # nan, -inf, +inf to numbers
    value = np.clip(value, min_value, max_value) # clip to valid
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def create_tfrecord(filename):
    with tempfile.TemporaryDirectory() as tmpdirname:
        TMPFILE="{}/read_grib".format(tmpdirname)
        tf.io.gfile.copy(filename, TMPFILE, overwrite=True)
        ds = xr.open_dataset(TMPFILE, engine='cfgrib', backend_kwargs={'filter_by_keys': {'typeOfLevel': 'atmosphere', 'stepType': 'instant'}})
   
        # create a TF Record with the raw data
        tfexample = tf.train.Example(
            features=tf.train.Features(
                feature={
                    'ref': _array_feature(ds.data_vars['refc'].data, min_value=0, max_value=60),
        }))
        return tfexample.SerializeToString()

s = create_tfrecord(FILENAME)
print(len(s), s[:16])

In [ ]:
from datetime import datetime, timedelta
def generate_filenames(startdate: str, enddate: str):
    start_dt = datetime.strptime(startdate, '%Y%m%d')
    end_dt = datetime.strptime(enddate, '%Y%m%d')
    dt = start_dt
    while dt <= end_dt:
        # gs://high-resolution-rapid-refresh/hrrr.20200811/conus/hrrr.t04z.wrfsfcf00.grib2
        f = '{}/hrrr.{:4}{:02}{:02}/conus/hrrr.t{:02}z.wrfsfcf00.grib2'.format(
                'gs://high-resolution-rapid-refresh',
                dt.year, dt.month, dt.day, dt.hour)
        dt = dt + timedelta(hours=1)
        yield f
        
def generate_shuffled_filenames(startdate: str, enddate: str):
    """
    shuffle the files so that a batch of records doesn't contain highly correlated entries
    """
    filenames = [f for f in generate_filenames(startdate, enddate)]
    np.random.shuffle(filenames)
    return filenames

print(generate_shuffled_filenames('20190915', '20190917'))

## Write a Beam pipeline

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%run -m wxsearch.hrrr_to_tfrecord -- --startdate 20190915 --enddate 20190916  --outdir gs://ai-analytics-solutions-kfpdemo/wxsearch/data --project ai-analytics-solutions
# --outdir tmp


In [ ]:
!gsutil ls gs://ai-analytics-solutions-kfpdemo/wxsearch/data/tfrecord*

## Read the written TF Records

In [ ]:
# try reading what was written out
import tensorflow as tf

def parse_tfrecord(example_data):
    parsed = tf.io.parse_single_example(example_data, {
        'size': tf.io.VarLenFeature(tf.float32),
        'ref': tf.io.VarLenFeature(tf.float32),
        'time': tf.io.FixedLenFeature([], tf.string),
        'valid_time': tf.io.FixedLenFeature([], tf.string)
     })
    parsed['size'] = tf.sparse.to_dense(parsed['size'])
    parsed['ref'] = tf.reshape(tf.sparse.to_dense(parsed['ref']), (1059, 1799))/60. # 0 to 1
    return parsed

def read_dataset(pattern):
    filenames = tf.io.gfile.glob(pattern)
    ds = tf.data.TFRecordDataset(filenames, compression_type=None, buffer_size=None, num_parallel_reads=None)
    return ds.prefetch(tf.data.experimental.AUTOTUNE).map(parse_tfrecord)

ds = read_dataset('gs://ai-analytics-solutions-kfpdemo/wxsearch/data/tfrecord-00000-*')
for refc in ds.take(1):
    print(repr(refc))

## Create autoencoder in Keras

In [ ]:
input_img = tf.keras.Input(shape=(1059, 1799, 1), name='refc_input')

x = tf.keras.layers.Cropping2D(cropping=((17, 18),(4, 3)), name='cropped')(input_img)
nlayers = 3
for layerno in range(nlayers):
    x = tf.keras.layers.Conv2D(2**(nlayers-layerno + 3), 5, activation='relu', padding='same', name='encoder_conv_{}'.format(layerno))(x)
    x = tf.keras.layers.MaxPooling2D(4, padding='same', name='encoder_pool_{}'.format(layerno))(x)
x = tf.keras.layers.Lambda(lambda x: x, name='refc_embedding')(x)
for layerno in range(nlayers):
    x = tf.keras.layers.Conv2D(2**(layerno + 4), 5, activation='relu', padding='same', name='decoder_conv_{}'.format(layerno))(x)
    x = tf.keras.layers.UpSampling2D(4, name='decoder_upsamp_{}'.format(layerno))(x)
x = tf.keras.layers.Conv2D(1, 3, activation='sigmoid', padding='same', name='before_padding')(x)
decoded = tf.keras.layers.ZeroPadding2D(padding=((17,18),(4,3)), name='refc_reconstructed')(x)

autoencoder = tf.keras.Model(input_img, decoded, name='autoencoder')
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

In [ ]:
tf.keras.utils.plot_model(autoencoder, to_file='autoencoder.png')

## Train the autoencoder

In [ ]:
def input_and_label(rec):
    return rec['ref'], rec['ref']

ds = read_dataset('gs://ai-analytics-solutions-kfpdemo/wxsearch/data/tfrecord-*').map(input_and_label).batch(2).repeat()
checkpoint = tf.keras.callbacks.ModelCheckpoint('tmp/checkpoints')
history = autoencoder.fit(ds, steps_per_epoch=1, epochs=3, shuffle=True, callbacks=[checkpoint])
print(history)

In [ ]:
autoencoder.save('tmp/savedmodel')

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss']);

In [ ]:
%%writefile wxsearch/train_autoencoder.py

import tensorflow as tf
import logging
import argparse
import os


def parse_tfrecord(example_data):
    parsed = tf.io.parse_single_example(example_data, {
        'size': tf.io.VarLenFeature(tf.float32),
        'ref': tf.io.VarLenFeature(tf.float32),
        'time': tf.io.FixedLenFeature([], tf.string),
        'valid_time': tf.io.FixedLenFeature([], tf.string)
     })
    parsed['size'] = tf.sparse.to_dense(parsed['size'])
    parsed['ref'] = tf.reshape(tf.sparse.to_dense(parsed['ref']), (1059, 1799))/60. # 0 to 1
    return parsed

def read_dataset(pattern):
    filenames = tf.io.gfile.glob(pattern)
    ds = tf.data.TFRecordDataset(filenames, compression_type=None, buffer_size=None, num_parallel_reads=None)
    return ds.prefetch(tf.data.experimental.AUTOTUNE).map(parse_tfrecord)

def create_model(nlayers=3, poolsize=4):
    input_img = tf.keras.Input(shape=(1059, 1799, 1), name='refc_input')

    x = tf.keras.layers.Cropping2D(cropping=((17, 18),(4, 3)), name='cropped')(input_img)
    for layerno in range(nlayers):
        x = tf.keras.layers.Conv2D(2**(nlayers-layerno + 3), 5, activation='relu', padding='same', name='encoder_conv_{}'.format(layerno))(x)
        x = tf.keras.layers.MaxPooling2D(poolsize, padding='same', name='encoder_pool_{}'.format(layerno))(x)
    x = tf.keras.layers.Lambda(lambda x: x, name='refc_embedding')(x)
    for layerno in range(nlayers):
        x = tf.keras.layers.Conv2D(2**(layerno + 4), 5, activation='relu', padding='same', name='decoder_conv_{}'.format(layerno))(x)
        x = tf.keras.layers.UpSampling2D(poolsize, name='decoder_upsamp_{}'.format(layerno))(x)
    x = tf.keras.layers.Conv2D(1, 3, activation='sigmoid', padding='same', name='before_padding')(x)
    decoded = tf.keras.layers.ZeroPadding2D(padding=((17,18),(4,3)), name='refc_reconstructed')(x)

    autoencoder = tf.keras.Model(input_img, decoded, name='autoencoder')
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

def run_job(opts):
    def input_and_label(rec):
        return rec['ref'], rec['ref']
    ds = read_dataset(opts['input']).map(input_and_label).batch(opts['batch_size']).repeat()
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(os.path.join(opts['outdir'], 'checkpoints'))
    
    autoencoder = create_model()
    history = autoencoder.fit(ds, steps_per_epoch=opts['num_steps']//opts['num_checkpoints'],
                              epochs=opts['num_checkpoints'], shuffle=True, callbacks=[checkpoint])
    
    autoencoder.save(os.path.join(opts['outdir'], 'savedmodel'))
    
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser(
      description='Train an autoencoder')
    parser.add_argument(
      '--project',
      default='',
      help='Specify GCP project to bill to run on cloud')
    parser.add_argument(
      '--outdir', required=True, help='output dir. could be local or on GCS')
    parser.add_argument(
      '--input', required=True, help='input pattern. eg: gs://ai-analytics-solutions-kfpdemo/wxsearch/data/tfrecord-*')
    parser.add_argument(
      '--batch_size', default=2, help='batch size for training')
    parser.add_argument(
      '--num_steps', default=12, help='total number of steps for training')
    parser.add_argument(
      '--num_checkpoints', default=3, help='number of steps for training')
     
     
    # parse command-line args and add a few more
    logging.basicConfig(level=getattr(logging, 'INFO', None))
    options = parser.parse_args().__dict__

    if not options['project']:
        print('Removing local output directory ... hang on')
        shutil.rmtree(outdir, ignore_errors=True)
        os.makedirs(outdir)
    else:
        print('Removing GCS output directory ... hang on')
        try:
            subprocess.check_call('gsutil -m rm -r {}'.format(outdir).split())
        except:  # pylint: disable=bare-except
            pass

    run_job(options)

In [ ]:
%autoreload 2
%run -m wxsearch.train_autoencoder -- --input gs://ai-analytics-solutions-kfpdemo/wxsearch/data/tfrecord-*  --outdir gs://ai-analytics-solutions-kfpdemo/wxsearch/trained --project ai-analytics-solutions

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License